In [21]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")
%aimport utils

import pandas as pd
import numpy as np

seed = 42
np.random.seed(seed)

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import Dense

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cancer_name = 'BRCA'
X_c, y_c = utils.get_cancer_data(cancer_name)
print("Cancer: {}".format(cancer_name))
print("\t#samples: {}".format(X_c.shape[0]))
print("\t#genes: {}".format(X_c.shape[1]))
print("\t#TUMORS: {}\t#NORMAL: {}".format(y_c[y_c == utils.TUMOR].shape[0], y_c[y_c == utils.NORMAL].shape[0]))

Cancer: BRCA
	#samples: 1218
	#genes: 20530
	#TUMORS: 1104	#NORMAL: 114


In [30]:
def get_random_genes(k, n):
    return np.random.choice(range(n), k, replace=False)

In [23]:
k = 400
n = X_c.shape[1]

times = 2

In [26]:
def get_filtered_features(X):
    return np.arange(X.shape[1])

def preprocess(X):
    scaler = MinMaxScaler()
    return utils.pre_process(X, get_filtered_features, scaler)

In [22]:
def net(input_size):
    """ A super-simple NN for the single tumor classification
    """
    model = Sequential()
    model.add(Dense(100, input_shape=(input_size,), activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [37]:
cvscoress_all = None

for i in range(times):
    rg = get_random_genes(k, n)
    #print(len(rg))
    X_c_i = X_c[:, rg]
    cvscores_c_i, histories_c_i = utils.cross_validation(X=X_c_i, y=y_c, 
                                                         preprocess=preprocess, 
                                                         seed=seed,
                                                         create_model=net, 
                                                         get_measures=utils.get_measures)
    cvscores_c_i['experiment'] = i
    cvscoress_all = pd.concat([cvscoress_all, cvscores_c_i], axis=0)

f1-score  0.92      precision 0.85      recall    1.00      accuracy  0.98      split     0.00      
f1-score  0.87      precision 0.77      recall    1.00      accuracy  0.97      split     1.00      
f1-score  0.94      precision 0.88      recall    1.00      accuracy  0.99      split     2.00      
f1-score  0.95      precision 1.00      recall    0.91      accuracy  0.99      split     3.00      
f1-score  0.94      precision 0.88      recall    1.00      accuracy  0.99      split     4.00      
f1-score  0.92      precision 0.85      recall    1.00      accuracy  0.98      split     0.00      
f1-score  0.96      precision 0.92      recall    1.00      accuracy  0.99      split     1.00      
f1-score  0.96      precision 0.92      recall    1.00      accuracy  0.99      split     2.00      
f1-score  0.98      precision 1.00      recall    0.96      accuracy  1.00      split     3.00      
f1-score  0.91      precision 0.91      recall    0.91      accuracy  0.98      split     4

In [40]:
cvscoress_all.to_csv("./results/random_selection.tsv", sep="\t", index=False, header=True)